# Exploring "HLSS30" using tensorlakehouse

### Data description
* Collection/dataset:"HLSS30"
* Data Source: GeoTiff stored on Cloud Object Store (COS) 

### Functionalities
* tensorlakehouse (openEO) functionalities:
    - describe collection
    - load collection

### Setup

1. Create a python virtualenv (python 3.11.7 is the recommended version). 
2. Clone the repository:
```
git clone https://github.com/IBM/tensorlakehouse-openeo-driver.git
``` 

3. Go to
```
cd tensorlakehouse-openeo-driver/tutorials
```

4. Install dependencies:
```
pip install -r tutorial_requirements.txt
```

5. Run jupyter notebook or jupyter lab
```
jupyter lab .
```


In [1]:
import rioxarray as rxr
import os
os.environ['USE_PYGEOS'] = '0'
import matplotlib.pyplot as plt
from pathlib import Path
# openeo is a client-side implementation that allows users to query OpenEO service
import openeo
# pystac_client is a client-side implementation that allows users to query STAC service
from pystac_client import Client
# OpenEO service URL
import os
from dotenv import load_dotenv
import pandas as pd
import xarray as xr
from shapely.geometry import shape
import geopandas
import folium
import time

Set URL and credentials of the OpenEO service

In [2]:
# username and password are hardcoded in the backend as an example for testing
load_dotenv()
STAC_URL="https://stac-fastapi-pgstac-geospatial-be-staging.apps.fmaas-backend.fmaas.res.ibm.com"
OPENEO_URL="https://tensorlakehouse-openeo-driver-geospatial-be-staging.apps.fmaas-backend.fmaas.res.ibm.com"

openeo_conn = openeo.connect(OPENEO_URL).authenticate_basic("john", "john123")
catalog = Client.open(STAC_URL)

### Setting input parameters: time range, area of interest, collection ID, filter

In [3]:
# set time interval
start ='2020-07-01T19:13:57Z'
end = '2020-07-01T20:13:57Z'
time_range = f"{start}/{end}"

# set bounding box
west = -123.0
east = -122.9
south = 38.0
north = 38.1

collection_id = "HLSS30"
# select items that have cloud_coverage field less than or equal to 50
filter_cql = {
                "op": "<",
                "args": [
                    {"property": "properties.cloud_coverage"},
                    "50",
                ],
            }
# language of the CQL2 query
filter_lang = "cql2-json"

# create data dir to save files
data_dir = Path("test_data")
if not data_dir.exists():
    data_dir.mkdir()

In [4]:


result = catalog.search(collections=[collection_id], bbox=[west, south, east, north], datetime=time_range, filter=filter_cql, filter_lang=filter_lang)
counter = 0
ids = list()
geometry = list()
timestamp = list()
bands = list()
cloud_coverage = list()
for item in result.items_as_dicts():
    ids.append(item["id"])
    geometry.append(shape(item["geometry"]))
    timestamp.append(pd.Timestamp(item["properties"]["datetime"]))
    bands.append(list(item["properties"]["cube:variables"].keys()))
    cloud_coverage.append(item["properties"]["cloud_coverage"])
gdf = geopandas.GeoDataFrame(data={"id": ids, "datetime": timestamp, "bands": bands, "cloud_coverage": cloud_coverage}, geometry=geometry)
gdf



,id,datetime,bands,cloud_coverage,geometry
0,HLS.S30.T10SDH.2020183T185919.v2.0.Fmask,2020-07-01 19:13:57.231648+00:00,[Fmask],19,"POLYGON ((-124.15275 38.84331, -124.13717 37.8..."
1,HLS.S30.T10SDH.2020183T185919.v2.0.B8A,2020-07-01 19:13:57.231648+00:00,[B8A],19,"POLYGON ((-124.15275 38.84331, -124.13717 37.8..."
2,HLS.S30.T10SDH.2020183T185919.v2.0.B12,2020-07-01 19:13:57.231648+00:00,[B12],19,"POLYGON ((-124.15275 38.84331, -124.13717 37.8..."
3,HLS.S30.T10SDH.2020183T185919.v2.0.B11,2020-07-01 19:13:57.231648+00:00,[B11],19,"POLYGON ((-124.15275 38.84331, -124.13717 37.8..."
4,HLS.S30.T10SDH.2020183T185919.v2.0.B10,2020-07-01 19:13:57.231648+00:00,[B10],19,"POLYGON ((-124.15275 38.84331, -124.13717 37.8..."
5,HLS.S30.T10SDH.2020183T185919.v2.0.B09,2020-07-01 19:13:57.231648+00:00,[B09],19,"POLYGON ((-124.15275 38.84331, -124.13717 37.8..."
6,HLS.S30.T10SDH.2020183T185919.v2.0.B08,2020-07-01 19:13:57.231648+00:00,[B08],19,"POLYGON ((-124.15275 38.84331, -124.13717 37.8..."
7,HLS.S30.T10SDH.2020183T185919.v2.0.B07,2020-07-01 19:13:57.231648+00:00,[B07],19,"POLYGON ((-124.15275 38.84331, -124.13717 37.8..."
8,HLS.S30.T10SDH.2020183T185919.v2.0.B06,2020-07-01 19:13:57.231648+00:00,[B06],19,"POLYGON ((-124.15275 38.84331, -124.13717 37.8..."
9,HLS.S30.T10SDH.2020183T185919.v2.0.B05,2020-07-01 19:13:57.231648+00:00,[B05],19,"POLYGON ((-124.15275 38.84331, -124.13717 37.8..."


In [5]:
minx, miny, maxx, maxy = gdf["geometry"].iloc[0].bounds

m = folium.Map(location=[(miny+maxy)/2, (minx+maxx)/2], zoom_start=6, control_scale=True)

# plot area of interest
aoi = {
      "type": "FeatureCollection",
      "features": [
        {
          "type": "Feature",
          "properties": {},
          "geometry": {
            "coordinates": [
              [
                [
                  west,
                  south
                ],
                [
                  west,
                  north
                ],
                [
                  east,
                  north
                ],
                [
                  east,
                  south
                ],
                [
                  west,
                  south
                ]
              ]
            ],
            "type": "Polygon"
          }
        }
      ]
    }

style_function = lambda x: {
    "color": "red",
    "fillColor": "red"
}
folium.GeoJson(aoi,
               style_function=style_function,
              ).add_to(m)

for geom in gdf["geometry"].unique():
    # selected_item = stac_df.iloc[i]
    # geom = selected_item["geometry"]
    lonmin, latmin, lonmax, latmax = geom.bounds
    # plot area of interest
    bbox = {
      "type": "FeatureCollection",
      "features": [
        {
          "type": "Feature",
          "properties": {},
          "geometry": {
            "coordinates": [
              [
                [
                  lonmin,
                  latmin
                ],
                [
                  lonmin,
                  latmax
                ],
                [
                  lonmax,
                  latmax
                ],
                [
                  lonmax,
                  latmin
                ],
                [
                  lonmin,
                  latmin
                ]
              ]
            ],
            "type": "Polygon"
          }
        }
      ]
    }


    folium.GeoJson(bbox,
                  # tooltip=folium.GeoJsonTooltip(fields=['FIELDNAME'])
                  ).add_to(m)
m

In [6]:
openeo_conn.describe_collection(collection_id)

{'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['B01', 'B02', 'B03', 'B04', 'B8A', 'B11', 'B12', 'Fmask']},
  'time': {'extent': ['2020-01-01T00:00:00+00:00',
    '2022-01-01T00:00:00+00:00'],
   'type': 'temporal'},
  'x': {'axis': 'x', 'extent': [399960.0, 509760.0], 'type': 'spatial'},
  'y': {'axis': 'y', 'extent': [4300020.0, 4190220.0], 'type': 'spatial'}},
 'deprecated': False,
 'description': 'HLSS30',
 'extent': {'spatial': {'bbox': [[-180.0, -90.0, 180.0, 90.0]]},
  'temporal': {'interval': [['2020-01-01T00:00:00Z',
     '2022-01-01T00:00:00Z']]}},
 'id': 'HLSS30',
 'keywords': [],
 'license': 'Unknown',
 'links': [{'href': 'http://stac-fastapi-pgstac.geospatial-be-staging.svc.cluster.local:8080/collections/HLSS30/items',
   'rel': 'items',
   'type': 'application/geo+json'},
  {'href': 'http://stac-fastapi-pgstac.geospatial-be-staging.svc.cluster.local:8080/',
   'rel': 'parent',
   'type': 'application/json'},
  {'href': 'http://stac-fastapi-pgstac.geospatial-be-staging.svc.cluster.local:8080/',
   'rel': 'root',
   'title': 'stac-fastapi',
   'type': 'application/json'},
  {'href': 'http://stac-fastapi-pgstac.geospatial-be-staging.svc.cluster.local:8080/collections/HLSS30',
   'rel': 'self',
   'type': 'application/json'}],
 'providers': [{'description': 'Producers of awesome spatiotemporal assets',
   'name': 'IBM',
   'roles': ['producer', 'processor'],
   'url': 'https://www.ibm.com'}],
 'stac_extensions': [],
 'stac_version': '1.0.0',
 'summaries': {},
 'title': None,
 'type': 'Collection',
 'version': ''}

## Loading tensorlakehouse datacube

In [7]:
bands = ["B02"]
spatial_extent={
        'west' : west,
        'south' : south,
        'east' : east,
        'north' : north
    }

temporal_extent = [start, end]

print(f"{collection_id=} {spatial_extent=} {temporal_extent=} {bands=}")
# note that 70 is a string because the data type of cloud_coverage is string
cube = openeo_conn.load_collection(
    collection_id=collection_id,
    spatial_extent=spatial_extent,
    temporal_extent=temporal_extent,
    bands=bands,
    properties=[
        openeo.collection_property("cloud_coverage") < "50",
    ]
)
# reproject cube to EPSG:4326 (note: resolution=0 means that no regridding is done)
cube = cube.resample_spatial(projection=4326, resolution=0)
# daily aggregation using minimum value
cube = cube.aggregate_temporal_period(
    period = "day",
    reducer = "min"
)

collection_id='HLSS30' spatial_extent={'west': -123.0, 'south': 38.0, 'east': -122.9, 'north': 38.1} temporal_extent=['2020-07-01T19:13:57Z', '2020-07-01T20:13:57Z'] bands=['B02']


/Users/ltizzei/.pyenv/versions/tensorlakehouse-openeo-tutorial/lib/python3.11/site-packages/openeo/rest/connection.py:1207: UserWarning: HLSS30 property filtering with properties that are undefined in the collection metadata (summaries): cloud_coverage.
  return DataCube.load_collection(


In [8]:
output_format = "netCDF"
cube = cube.save_result(output_format)

### Submit a batch job and wait until it finishes

In [ ]:
# create a batch jobs
job = cube.create_job(out_format=output_format)
job_status = job.status()
job_id = job.job_id
# create a directory to store the results
output_dir = data_dir / f"test_batch_jobs_{job_id}"
if not output_dir.exists():
    output_dir.mkdir()
# while jobs is still in progress, wait for it
while job_status not in ["canceled", "finished", "error"]:
    time.sleep(30)
    job_status = job.status()
    print(f"{job_status=}")
# get results
results = job.get_results()
# download files
results.download_files(output_dir)
print(f"Downloaded to {output_dir} directory")

Preflight process graph validation failed: [403] AuthenticationSchemeInvalid: Authentication method not supported. (ref: r-2406241ff0a44151b6c2a2e2a13add34)


### List files, open the netcdf file and plot the image

In [ ]:
downloaded_files = list(output_dir.glob("*"))
downloaded_files

In [ ]:
# listing all netcdf files
netcdf_files = [f for f in downloaded_files if f.suffix == ".nc"]
ds = xr.open_dataset(netcdf_files[0])
ds

In [ ]:
ds.time.values

In [ ]:
# open all raster files and concatenate

da = ds[bands[0]]
da.isel({"time":0}).plot()